# Todo

- [x] 把成交量整理成每週加總
- [ ] 加總同縣市市場

In [1]:
import pandas as pd

# Load Data

In [12]:
total_data = pd.read_csv('../price_with_typhoon_utf8.csv')
total_data = total_data[['DateTime', '交易量', '市場名稱', '作物名稱']]

In [13]:
USED_CROPS = ['鳳梨-金鑽鳳梨', '香蕉', '青蔥-日蔥', '蓮霧-紅蓮霧', '芽菜類-黃豆牙', '南瓜-木瓜形', '大蒜-蒜仁', '小白菜-土白菜', '木瓜-網室紅肉', '洋香瓜-網狀紅肉']
total_data = total_data[total_data['作物名稱'].isin(USED_CROPS)]

# Sum up the total volume week by week

## Create weekly sequence

In [18]:
date_points = []
start_date = pd.to_datetime('2012-01-01') - pd.DateOffset(days=7)
end_date = start_date

while end_date < pd.to_datetime('2020-01-01'):
  start_date = end_date
  end_date = pd.to_datetime(start_date) + pd.DateOffset(days=7)
  date_points.append((start_date, end_date))

## Extract 7 days from the data

In [21]:
START = 0
END = 1

def extract_period(df, start, end):
  return df[(pd.to_datetime(df['DateTime']) >= start) & (pd.to_datetime(df['DateTime']) < end)]

## Group by crop & market and sum up the volume

In [53]:
def sum_up_week(df, start, end):
  summary = {'作物名稱': [], '市場名稱': [], '交易量': [], 'DateTime': [], 'Period': []}
  week_data = extract_period(df, start, end)
  week_sum = week_data.groupby(by=['作物名稱','市場名稱']).sum()
  
  for name, row in week_sum.iterrows():
    summary['作物名稱'].append(name[0])
    summary['市場名稱'].append(name[1])
    summary['交易量'].append(row.values[0])
    summary['DateTime'].append(str(start.date()))
    summary['Period'].append(str(start.date())+'~'+str(end.date()))
  
  return pd.DataFrame.from_dict(summary)

In [56]:
total_weeks = []

for d in date_points:
  total_weeks.append(sum_up_week(total_data, d[START], d[END]))

In [59]:
# save file
pd.concat(total_weeks).to_csv('volume_per_week.csv', index=False)